In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
from datetime import datetime

# ==================== CONFIGURACIÓN ====================

# Parámetros fijos
DEPARTAMENTO = "LIMA"
PROVINCIA = "LIMA"
RESULTADOS = "50"

# XPaths
XPATH_SELECT_DEPTO = '//*[@id="contact"]/div/div[1]/div/div[1]/select'
XPATH_SELECT_PROV  = '//*[@id="contact"]/div/div[1]/div/div[2]/select'
XPATH_SELECT_DIST  = '//*[@id="contact"]/div/div[1]/div/div[3]/select'
XPATH_SELECT_PROD  = '//*[@id="contact"]/div/div[1]/div/div[4]/select'
XPATH_LEN_TABLE    = '//*[@id="tblPreciosAutomotor_length"]/label/select'
XPATH_TABLE        = '//*[@id="tblPreciosAutomotor"]'
URL = "https://www.facilito.gob.pe/facilito/pages/facilito/buscadorEESS.jsp"

def click_center_of_viewport(driver):
    js = """
    const x = Math.floor(window.innerWidth / 2);
    const y = Math.floor(window.innerHeight / 2);
    const el = document.elementFromPoint(x, y);
    if (el) {
        const ev = new MouseEvent('click', {bubbles: true, clientX: x, clientY: y});
        el.dispatchEvent(ev);
        return true;
    }
    return false;
    """
    driver.execute_script(js)

def gather_data(productos, folder_suffix, all_data):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")

    
    # --- MODO HEADLESS (oculta la ventana) ---
    options.add_argument("--headless=new")  # para Chrome moderno
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 20)

    try:
        driver.get(URL)
        time.sleep(2)

        click_center_of_viewport(driver)
        time.sleep(2)

        # 1️⃣ Departamento
        dept = wait.until(EC.presence_of_element_located((By.XPATH, XPATH_SELECT_DEPTO)))
        Select(dept).select_by_visible_text(DEPARTAMENTO)
        time.sleep(2)

        # 2️⃣ Provincia
        prov = wait.until(EC.presence_of_element_located((By.XPATH, XPATH_SELECT_PROV)))
        Select(prov).select_by_visible_text(PROVINCIA)
        time.sleep(2)

        # 3️⃣ Obtener lista completa de distritos disponibles
        dist_elem = wait.until(EC.presence_of_element_located((By.XPATH, XPATH_SELECT_DIST)))
        all_distritos = [opt.text.strip() for opt in dist_elem.find_elements(By.TAG_NAME, "option") if opt.text.strip() and "Seleccione" not in opt.text]
        # print(f"🔍 Distritos encontrados: {len(all_distritos)} → {all_distritos}")

        # 4️⃣ Iterar sobre cada distrito
        for distrito in all_distritos[1:2]:  # Limitar a los primeros 5 distritos para pruebas
            dist_elem = wait.until(EC.presence_of_element_located((By.XPATH, XPATH_SELECT_DIST)))
            Select(dist_elem).select_by_visible_text(distrito)
            time.sleep(2)

            for producto in productos:
                prod_elem = wait.until(EC.presence_of_element_located((By.XPATH, XPATH_SELECT_PROD)))
                Select(prod_elem).select_by_visible_text(producto)
                time.sleep(3)

                # Mostrar 50 resultados si está disponible
                try:
                    len_select = wait.until(EC.presence_of_element_located((By.XPATH, XPATH_LEN_TABLE)))
                    Select(len_select).select_by_visible_text(RESULTADOS)
                    time.sleep(2)
                except Exception:
                    pass

                # Obtener tabla
                try:
                    table_elem = wait.until(EC.presence_of_element_located((By.XPATH, XPATH_TABLE)))
                    table_html = table_elem.get_attribute("outerHTML")
                    tables = pd.read_html(table_html)

                    if len(tables) > 0:
                        df = tables[0]
                        df["Departamento"] = DEPARTAMENTO
                        df["Provincia"] = PROVINCIA
                        df["Distrito"] = distrito
                        df["Producto"] = producto
                        all_data.append(df)  # Agregar el DataFrame al conjunto de datos
                        print(f"✅ ({all_distritos.index(distrito)}/{len(all_distritos)}) {distrito} | {producto}: {len(df)} filas capturadas")
                    else:
                        print(f"⚠️ {distrito} | {producto}: tabla vacía")

                except Exception as e:
                    print(f"⚠️ Error capturando datos en {distrito}: {e}")

    finally:
        driver.quit()

# Lista para almacenar todos los datos
all_data = []

# Parámetros para cada producto
productos_1 = ["DB5 S-50 UV"]
productos_2 = ["Gasohol Premium"]
productos_3 = ["Gasohol Regular"]

# Llamadas a la función para cada caso y almacenar en all_data
gather_data(productos_1, "DB5_UV", all_data)
gather_data(productos_2, "GasPremium", all_data)
gather_data(productos_3, "GasRegular", all_data)


######################################################################################
######################################################################################
######################################################################################
######################################################################################


today = datetime.now()

final_df = pd.concat(all_data, ignore_index=True)
final_df["Fecha"] = today.date()
final_df['Precio'] = pd.to_numeric(final_df['Precio de Venta (Soles por galón)'], errors="coerce")

#FOLDER
folder_name = f"data/raw/osinergmin/"
os.makedirs(folder_name, exist_ok=True)

# Ruta del archivo consolidado
file_path = os.path.join(folder_name, f"osinergmin_{today.strftime('%d_%m_%Y')}.csv")

final_df.to_csv(file_path, index=False, encoding="utf-8-sig")
# final_df

In [ ]:
all_distritos

In [ ]:

today = datetime.now()

final_df = pd.concat(all_data, ignore_index=True)
final_df["Fecha"] = today.date()
final_df['Precio'] = pd.to_numeric(final_df['Precio de Venta (Soles por galón)'], errors="coerce")

#FOLDER
folder_name = f"data/raw/osinergmin/"
os.makedirs(folder_name, exist_ok=True)

# Ruta del archivo consolidado
file_path = os.path.join(folder_name, f"osinergmin_{today.strftime('%d_%m_%Y')}.csv")

final_df.to_csv(file_path, index=False, encoding="utf-8-sig")
final_df

In [ ]:
import pandas as pd
import glob

In [ ]:
# Carpeta - data diaria
ruta = "data/raw/osinergmin/"  
archivos = sorted(glob.glob(ruta + "osinergmin_*.csv"))

print(f"Archivos: {len(archivos)}")

dfs = []

for archivo in archivos:
    try:
        df = pd.read_csv(archivo)
        df["__origen__"] = archivo  # para rastrear el archivo
        dfs.append(df)
    except Exception as e:
        print(f"Error leyendo {archivo}: {e}")

# Concatenar todo
osinerg = pd.concat(dfs, ignore_index=True)

# Remover duplicados por si existen repeticiones
osinerg = osinerg.drop_duplicates()
# osinerg.to_csv("data/consolidated/combustibles/osinergmin.csv", index=False)
osinerg.head(4)


In [ ]:
osinerg[osinerg["Producto"]=="DB5 S-50 UV"].Precio.hist(bins=100)
osinerg[osinerg["Producto"]=="Gasohol Regular"].Precio.hist(bins=100)
osinerg[osinerg["Producto"]=="Gasohol Premium"].Precio.hist(bins=100)

In [ ]:
import matplotlib.pyplot as plt

# --- 1. Un solo gráfico con los tres histogramas juntos ---
plt.figure(figsize=(10,6))

osinerg[osinerg["Producto"]=="DB5 S-50 UV"]["Precio"].hist(bins=100, alpha=0.5)
osinerg[osinerg["Producto"]=="Gasohol Regular"]["Precio"].hist(bins=100, alpha=0.5)
osinerg[osinerg["Producto"]=="Gasohol Premium"]["Precio"].hist(bins=100, alpha=0.5)

plt.title("Histogramas juntos")
plt.xlabel("Precio")
plt.ylabel("Frecuencia")
plt.legend(["DB5 S-50 UV", "Gasohol Regular", "Gasohol Premium"])
plt.show()

In [ ]:
df = osinerg

In [ ]:
import numpy as np

In [ ]:
# df["Fecha"] = pd.to_datetime(df["Fecha"])

# --- Agrupar por día con PROMEDIO GEOMÉTRICO + STD ---
daily = df.groupby("Fecha").agg(
    promedio_geom = ("Precio", lambda x: np.exp(np.log(x).mean())),
    volatilidad   = ("Precio", "std")
).reset_index()

# Renombrar para que todo use el promedio geométrico
daily["precio_base"] = daily["promedio_geom"]

# Bandas geom ± std
daily["banda_sup"] = daily["precio_base"] + daily["volatilidad"]
daily["banda_inf"] = daily["precio_base"] - daily["volatilidad"]

# Orden
daily = daily.sort_values("Fecha").reset_index(drop=True)

# --- Base 100 usando el 29-10-2025 ---
base_fecha = pd.to_datetime("2025-10-29")

base_valor = daily.loc[daily["Fecha"] == base_fecha, "precio_base"]

if base_valor.empty:
    raise ValueError("No existe información para la fecha base 29-10-2025.")

base_valor = base_valor.iloc[0]

# Índices base 100
daily["idx"]     = 100 * daily["precio_base"] / base_valor
daily["idx_sup"] = 100 * daily["banda_sup"]   / base_valor
daily["idx_inf"] = 100 * daily["banda_inf"]   / base_valor

daily.head()


In [ ]:
daily

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Línea principal
fig.add_trace(go.Scatter(
    x=daily["Fecha"],
    y=daily["idx"],
    mode="lines",
    name="Índice (Base 100)",
    line=dict(width=2)
))

# # Banda superior
# fig.add_trace(go.Scatter(
#     x=daily["Fecha"],
#     y=daily["idx_sup"],
#     mode="lines",
#     name="Banda superior",
#     line=dict(width=1, dash="dot")
# ))

# # Banda inferior
# fig.add_trace(go.Scatter(
#     x=daily["Fecha"],
#     y=daily["idx_inf"],
#     mode="lines",
#     name="Banda inferior",
#     line=dict(width=1, dash="dot")
# ))

# # Relleno entre bandas
# fig.add_trace(go.Scatter(
#     x=pd.concat([daily["Fecha"], daily["Fecha"][::-1]]),
#     y=pd.concat([daily["idx_sup"], daily["idx_inf"][::-1]]),
#     fill="toself",
#     fillcolor="rgba(0, 100, 200, 0.15)",
#     line=dict(color="rgba(0,0,0,0)"),
#     hoverinfo="skip",
#     showlegend=False
# ))

fig.update_layout(
    title="Índice Diario (Base 100 = 29/10/2025) usando Promedio Geométrico",
    xaxis_title="Fecha",
    yaxis_title="Índice (Base 100)",
    template="plotly_white",
    hovermode="x unified",
    width=900,
    height=500
)

fig.show()
